In [10]:
import os
import pickle

import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from transformers import BertModel, BertTokenizer
import re
import csv   

import sys
sys.path.append("..")
from src.bert import *


def get_context(token_ids, target_position, sequence_length=128):
    """
    Given a text containing a target word, return the sentence snippet which surrounds the target word
    (and the target word's position in the snippet).

    :param token_ids: list of token ids (for an entire line of text)
    :param target_position: index of the target word's position in `tokens`
    :param sequence_length: desired length for output sequence (e.g. 128, 256, 512)
    :return: (context_ids, new_target_position)
                context_ids: list of token ids for the output sequence
                new_target_position: index of the target word's position in `context_ids`
    """
    # -2 as [CLS] and [SEP] tokens will be added later; /2 as it's a one-sided window
    window_size = int((sequence_length - 2) / 2)
    context_start = max([0, target_position - window_size])
    padding_offset = max([0, window_size - target_position])
    padding_offset += max([0, target_position + window_size - len(token_ids)])

    context_ids = token_ids[context_start:target_position + window_size]
    context_ids += padding_offset * [0]

    new_target_position = target_position - context_start

    return context_ids, new_target_position

def get_usage_vectors( model,
                       batch_input_ids,
                       batch_tokens,
                       batch_snippets,
                       batch_pos, 
                       batch_decades):
    
    with torch.no_grad():
        # collect list of input ids into a single batch tensor
        input_ids_tensor = torch.tensor(batch_input_ids)
        if torch.cuda.is_available():
            input_ids_tensor = input_ids_tensor.to('cuda')

            
        #print(input_ids_tensor)
        # run usages through language model
        outputs = model(input_ids_tensor,  output_hidden_states=True )
        #print(len(outputs.hidden_states)) # items in the tuple = 1 + num layers
        if torch.cuda.is_available():
            hidden_states = [l.detach().cpu().clone().numpy() for l in outputs[2]]
        else:
            #print("fjekl")
            hidden_states = [l.clone().numpy() for l in outputs.hidden_states]

        # get usage vectors from hidden states
        hidden_states = np.stack(hidden_states)  # (13, B, |s|, 768)
        #print('Expected hidden states size: (13, B, |s|, 768). Got {}'.format(hidden_states.shape))
        # usage_vectors = np.sum(hidden_states, 0)  # (B, |s|, 768)
        # usage_vectors = hidden_states.view(hidden_states.shape[1],
        #                                    hidden_states.shape[2],
        #                                    -1)
        usage_vectors = np.sum(hidden_states[1:, :, :, :], axis=0)
        
        # usage_vectors = hidden_states.reshape((hidden_states.shape[1], hidden_states.shape[2], -1))
        #print(usage_vectors.shape)
        return usage_vectors
    
def get_feature_vectors( model,
                        feature_model,
                       batch_input_ids,
                       batch_tokens,
                       batch_snippets,
                       batch_pos, 
                       batch_decades):
    
    with torch.no_grad():
        # collect list of input ids into a single batch tensor
        input_ids_tensor = torch.tensor(batch_input_ids)
        if torch.cuda.is_available():
            input_ids_tensor = input_ids_tensor.to('cuda')

            
        #print(input_ids_tensor)
        # run usages through language model
        outputs = model(input_ids_tensor,  output_hidden_states=True )
        #print(len(outputs.hidden_states)) # items in the tuple = 1 + num layers
        if torch.cuda.is_available():
            hidden_states = [l.detach().cpu().clone().numpy() for l in outputs[2]]
        else:
            #print("fjekl")
            hidden_states = [l.clone().numpy() for l in outputs.hidden_states]

        # get usage vectors from hidden states
        hidden_states = np.stack(hidden_states)  # (13, B, |s|, 768)
        print('Expected hidden states size: (13, B, |s|, 768). Got {}'.format(hidden_states.shape))
        # usage_vectors = np.sum(hidden_states, 0)  # (B, |s|, 768)
        # usage_vectors = hidden_states.view(hidden_states.shape[1],
        #                                    hidden_states.shape[2],
        #                                    -1)
        
        #print(hidden_states.shape)
        #print(hidden_states[1:, :, :, :].shape)
        usage_vectors = hidden_states[8, :, :, :] # get the 8th layer, which the model expects as input
        
        # usage_vectors = hidden_states.reshape((hidden_states.shape[1], hidden_states.shape[2], -1))
        #print(usage_vectors.shape)
        
        
        
        return usage_vectors


def collect_from_coha(target_words,
                      decades,
                      sequence_length,
                      pretrained_weights='models/bert-base-uncased',
                      coha_dir='data/coha',
                      output_path=None,
                      buffer_size=1024,
                      feature_model=None):
    """
    Collect usages of target words from the COHA dataset.

    :param target_words: list of words whose usages are to be collected
    :param decades: list of year integers (e.g. list(np.arange(1910, 2001, 10)))
    :param sequence_length: the number of tokens in the context of a word occurrence
    :param pretrained_weights: path to model folder with weights and config file
    :param coha_dir: path to COHA directory (containing `all_1810.txt`, ..., `all_2000.txt`)
    :param output_path: path to output file for `usages` dictionary. If provided, data is stored
                        in this file incrementally (use e.g. to avoid out of memory errors)
    :param buffer_size: (max) number of usages to process in a single model run
    :return: usages: a dictionary from target words to lists of usage tuples
                     lemma -> [(vector, sentence, word_position, decade), (v, s, p, d), ...]
    """

    # load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    model = BertModel.from_pretrained(pretrained_weights)
    if torch.cuda.is_available():
        model.to('cuda')
        
        
    feature_model = torch.load(feature_model)


    # build word-index vocabulary for target words
    i2w = {}
    for t, t_id in zip(target_words, tokenizer.encode(' '.join(target_words))[1:-1]): #changed for new hugginface api
        i2w[t_id] = t

    # buffers for batch processing
    batch_input_ids = []
    batch_tokens = []
    batch_pos = []
    batch_snippets = []
    batch_decades = []

    usages = defaultdict(list)  # w -> (vector, sentence, word_position, decade)
    
    # do collection
    
    print(len(decades))
    for T, decade in enumerate(decades):
        # one time interval at a time
        print('Decade {}...'.format(decade))

        
        ### gabriella changes
        ### my coha is organized differently. 
        ### the decades have random numbers for the alphabet index places , so i have to use regex
        ### to ignore that. 
        print(coha_dir)
        print(decade)
        my_regex = r'text_' + re.escape(str(decade)) + 's.*'

        #print("running through decade ", decade)

        # iterate through directories
        for decade_dir in os.listdir(coha_dir):

            if re.match(my_regex, decade_dir):
                # get all the text files for that decade
                # iterate through text files for this decade
                this_decade_files = os.listdir(os.path.join(coha_dir, decade_dir))
                for F, filename in enumerate(tqdm(this_decade_files)):
                    #print(filename)
                    
                    with open(os.path.join(coha_dir, decade_dir, filename), 'r') as f:
                        lines = f.readlines()
                        #print("gets here")

                        # get the usages from this file
                        for L, line in enumerate(lines):
                            #print("gets to line: ", L)



                            # tokenize line and convert to token ids
                            tokens = tokenizer.encode(line)

                            for pos, token in enumerate(tokens):
                                #print(token)
                                # store usage info of target words only
                                if token in i2w:
                                    context_ids, pos_in_context = get_context(tokens, pos, sequence_length)

                                    input_ids = [101] + context_ids + [102]


                                    # convert later to save storage space
                                    snippet = tokenizer.convert_ids_to_tokens(context_ids)
                                    #print(i2w[token])
                                    #print(' '.join(snippet))

                                    # add usage info to buffers
                                    batch_input_ids.append(input_ids)
                                    batch_tokens.append(i2w[token])
                                    batch_pos.append(pos_in_context)
                                    batch_snippets.append(snippet)
                                    batch_decades.append(decade)

#                                 print("batch size ", len(batch_input_ids))
#                                 print("lines left ", len(lines) - L)
#                                 print("files left in this decade" , len(this_decade_files) - F)
#                                 print("decades left", len(decades) - T)
                                    
                                # if the buffers are full...             or if we're at the end of the dataset
                                if (len(batch_input_ids) >= buffer_size) or (L == len(lines) - 1 and T == len(decades) - 1 and F==len(this_decade_files)):
                                    
                                    usage_vectors = get_feature_vectors(
                                        model,
                                        feature_model,
                                        batch_input_ids,
                                        batch_tokens,
                                        batch_snippets, 
                                        batch_pos, 
                                        batch_decades )


                                    # store usage tuples in a dictionary: lemma -> (vector, snippet, position, decade)
                                    #print(len(batch_input_ids))
                                    for b in np.arange(len(batch_input_ids)):
                                        #print(usage_vectors.shape)
                                        layer_8_bert_vector = usage_vectors[b, batch_pos[b]+1, :] # get the right position
                                        
                                        word, feature_vector = feature_model.predict_from_single_context_vector(batch_tokens[b], layer_8_bert_vector)
                                        
                                        usages[batch_tokens[b]].append(
                                             (feature_vector, batch_snippets[b], batch_pos[b], batch_decades[b]))
                                    
#                                     print(usages)
                        
                                    # finally, empty the batch buffers
                                    batch_input_ids, batch_tokens, batch_pos, batch_snippets, batch_decades = [], [], [], [], []
                        
        print("saving usages for decade")
        if os.path.exists(output_path):
            append_write = 'ab' # append if already exists
        else:
            append_write = 'wb' # make a new file if not

        # and store data incrementally
        if output_path:
            #print(append_write)
            with open(output_path, append_write) as f:
                pickle.dump(usages, file=f)
    

    return usages


In [11]:
# Target words: we want to collect tokens of each of these words from COHA

targets = ['net', 'virtual', 'disk', 'card', 'optical', 'virus',
           'signal', 'mirror', 'energy', 'compact', 'leaf',
           'brick', 'federal', 'sphere', 'coach', 'spine', 'parent', 'sleep', 'fire']

decades = [decade for decade in np.arange(1910, 2009, 10)]
print(len(decades))

buffer_size=1024
sequence_length=128


coha_dir = '/home/shared/corpora/Corpus of Historical American English/TEXTS'
bert_dir = 'bert-base-uncased'
output_dir = '../data/cwr4lsc/binder'
feature_model = '../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300'

10


In [ ]:
### collect just the usages and not the vectors. 

for decade in decades:
    collect_from_coha(targets,
                      [decade],
                      sequence_length=sequence_length,
                      pretrained_weights=bert_dir,
                      coha_dir=coha_dir,
                      output_path='{}/usages_with_vectors_16_len{}_{}.dict'.format(output_dir, sequence_length, decade),
                      buffer_size=buffer_size,
                      feature_model=feature_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1
Decade 1910...
/home/shared/corpora/Corpus of Historical American English/TEXTS
1910


  7%|███████████▊                                                                                                                                                            | 237/3355 [00:14<02:44, 18.90it/s]

Expected hidden states size: (13, B, |s|, 768). Got (13, 1024, 128, 768)


 14%|████████████████████████▎                                                                                                                                               | 486/3355 [00:34<03:57, 12.10it/s]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                      | 9309/13795 [05:03<04:18, 17.34it/s]

Expected hidden states size: (13, B, |s|, 768). Got (13, 1024, 128, 768)


 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                   | 9532/13795 [05:19<02:13, 31.85it/s]

Expected hidden states size: (13, B, |s|, 768). Got (13, 1024, 128, 768)


 70%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                  | 9644/13795 [05:26<02:27, 28.16it/s]

In [ ]:
"""
It's finished! we got the binder vectors.
"""

In [4]:
binder_model = torch.load(feature_model)

tokenizer = BertTokenizer.from_pretrained(bert_dir)
model = BertModel.from_pretrained(bert_dir)


encoded_input = tokenizer("hello", return_tensors='pt')
outputs = model(**encoded_input, output_hidden_states=True)
hidden_states = [l.detach().cpu().clone().numpy() for l in outputs[2]]



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
vec = hidden_states

In [6]:
vec[8][0][0].shape

(768,)

In [7]:
binder_model.predict_from_single_context_vector("hello", vec[8][0][0])

('hello',
 array([ 1.0056549e+00,  3.8931992e-02, -3.6725351e-01, -9.3476987e-01,
        -2.7887559e-01,  7.2497123e-01, -2.0921183e-01,  9.4847614e-01,
         1.7737154e+00,  3.1238947e-02, -7.7192754e-02, -1.0627046e-01,
         1.7184262e+00,  2.0627036e+00,  4.8392379e-01,  2.0409809e-01,
        -6.2202502e-02, -5.3744036e-01,  7.2529328e-01,  2.8303607e+00,
         2.0169008e+00,  5.7054991e-01,  1.5627507e+00,  2.5735607e+00,
         1.1967081e+00,  1.9624131e+00, -2.8737587e-01,  4.0165412e-01,
         1.9263399e+00,  5.1853997e-01,  2.5976652e-01,  1.6036888e+00,
        -6.2239653e-01,  1.0072670e+00,  1.5316712e+00,  8.3211809e-05,
        -1.4192641e-01,  1.7934600e-01,  1.5579345e-02,  8.6351700e-02,
         3.7302986e-01,  5.3672683e-01,  1.1006322e+00,  1.4275689e+00,
         1.5960541e+00,  1.4706843e+00,  1.4171218e+00,  4.3489739e-01,
         1.8168885e+00,  5.2834034e-01,  1.7661784e+00,  4.5286259e-01,
         1.6958729e+00,  1.9344276e-01,  4.2212194e-01

In [8]:
buchanan = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')


/home/gsc685/.conda/envs/gabriella_cwr4lsc/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PLSRegression from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
buchanan.predict_from_single_context_vector("hello", vec[8][0][0])

('hello',
 array([-0.00748495, -0.00382117, -0.05292879, ...,  0.00085026,
         0.00424264,  0.0031281 ]))